Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License. 

# MONAI Auto3DSeg Hyper-parameter Optimization with Optuna

This notebook provides an example to perform hyperparameter optimization(HPO) on learning rate with grid search method for hippocampus segmentation using Optuna.

Note: if you have used other Auto3DSeg notebooks on the hippocampus dataset (MSD Task 04), for example: 
- `auto_runner.ipynb`
- `auto3dseg_autorunner_ref_api.ipynb`
- `auto3dseg_hello_world.ipynb`
- `hpo_nni.ipynb`

You may have already generated the algorithm templates in MONAI bundle formats (hint: find them in the working directory). 

## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel, tqdm, cucim, yaml, optuna, nni]"

## Setup imports

In [ ]:
import os
import optuna
import tempfile
import torch

from functools import partial

from monai.apps import download_and_extract
from monai.apps.auto3dseg import BundleGen, DataAnalyzer, OptunaGen
from monai.apps.auto3dseg.utils import export_bundle_algo_history, import_bundle_algo_history
from monai.bundle.config_parser import ConfigParser
from monai.config import print_config

print_config()

## Download dataset

We provide a toy datalist file that splits a subset of the downloaded datasets into five folds.

> NOTE: Each validation set only has 6 images in one fold of training.
> Therefore, we need to set a limit on the total number of GPUs we're using in this notebook.

In [ ]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

msd_task = "Task04_Hippocampus"
resource = "https://msd-for-monai.s3-us-west-2.amazonaws.com/" + msd_task + ".tar"

compressed_file = os.path.join(root_dir, msd_task + ".tar")
dataroot = os.path.join(root_dir, msd_task)
if not os.path.exists(dataroot):
    download_and_extract(resource, compressed_file, root_dir)

datalist_file = os.path.join("..", "tasks", "msd", msd_task, "msd_" + msd_task.lower() + "_folds.json")

if torch.cuda.device_count() > 6:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5"

## Define experiment file pathes

In [ ]:
# Experiment setup
work_dir = "./hpo_optuna_work_dir"
datastats_file = os.path.join(work_dir, "datastats.yaml")
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

## Prepare an input YAML

This is an example for the dataset.
User should generate their own.

In [ ]:
input_cfg = {
    "name": msd_task,  # optional, it is only for your own record
    "task": "segmentation",  # optional, it is only for your own record
    "modality": "MRI",  # required
    "datalist": datalist_file,  # required
    "dataroot": dataroot,  # required
}
input = "./input.yaml"
ConfigParser.export_config_file(input_cfg, input)

## Create Bundle Generators


In [ ]:
if not os.path.exists(datastats_file):
    da = DataAnalyzer(datalist_file, dataroot, output_path=datastats_file)
    da.get_all_case_stats()

# algorithm generation
bundle_generator = BundleGen(
    algo_path=work_dir,
    data_stats_filename=datastats_file,
    data_src_cfg_name=input,
)

bundle_generator.generate(work_dir, num_fold=1)
history = bundle_generator.get_history()
export_bundle_algo_history(history)

## Create Algo object from bundle_generator history

Algorithm selected to do HPO.
Refer to bundle history for the mapping between algorithm name and index.
Users can `get_history` from `bundle_generator`, or `import_bundle_algo_history` by reading bundles saved in the working directory

In [ ]:
try:
    history = bundle_generator.get_history()
    assert len(history) > 0
except Exception:
    history = import_bundle_algo_history(work_dir, only_trained=False)

print("algorithms imported from the history:")
for i, algo_dict in enumerate(history):
    print(f"{i}: ", list(algo_dict.keys())[0])

## Select an algorithm to perform HPO
Note: The name of the algorithms has a convention: `{net}_{fold_index}_{other_meta_info}`

In [ ]:
selected_algorithm_index = 0
algo_dict = history[selected_algorithm_index]
algo_name = list(algo_dict.keys())[0]
algo = algo_dict[algo_name]
print("User selected algorithm: ")
print(algo_name)

## (Optional) Override params to complete training and HPO for demo purpose

Users can override the parameters of choice before the HPO runs.
In such case, users need to provide an `override_param`.
The previously-generated algorithm will not be touched.
`NNIGen` will create a copy of the algorithm and save the algorithm in a new folder named `{net}_{fold_index}_override`.

For more information about creating overriding parameters, please refer to the section "Override Specific Parameters in the Algorithms before HPO" in this [tutorial documentation](../docs/hpo.md)

> NOTE: The setup works fine for a machine that has GPUs less than or equal to 8.
> The datalist in this example is only using a subset of the original dataset.
> Users need to ensure the number of GPUs is not greater than the number that the training dataset can be partitioned.
> For example, the following code block is not suitable for a 16-GPU system.
> In such cases, please change the code block accordingly.

In [ ]:
# "override_params" is used to update algorithm hyperparameters
# like num_epochs, which are not in the HPO search space. We set num_epochs=2
# to shorten the training time as an example

max_epochs = 2

# for segresnet2d
override_param = {
    "num_epochs_per_validation": 1,
    "num_epochs": max_epochs,
}

## Create Optuna Generator class and overwrite get_hyperparameters() function

In [ ]:
class OptunaGenLearningRate(OptunaGen):
    def get_hyperparameters(self):
        return {"learning_rate": self.trial.suggest_float("learning_rate", 0.00001, 0.1)}


optuna_gen = OptunaGenLearningRate(algo=algo, params=override_param)

## Run Optuna optimization (with grid search)

In [ ]:
search_space = {"learning_rate": [0.0001, 0.001, 0.01, 0.1]}
study = optuna.create_study(sampler=optuna.samplers.GridSampler(search_space), direction="maximize")
study.optimize(partial(optuna_gen, obj_filename=optuna_gen.get_obj_filename(), output_folder=work_dir), n_trials=2)
print("Best value: {} (params: {})\n".format(study.best_value, study.best_params))